In [ ]:
import pandas as pd
import dateutil
import pylab as plt
import seaborn as sns
from matplotlib import dates as mdates
import cdflib 
import numpy as np

%matplotlib inline
sns.set_style('dark')

In [ ]:
SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 22
DPI = 500

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
def format_title(filename):
    if 'RCM' in filename:
        return 'LFM-RCM'
    else:
        return 'LFM'

def format_key(key):
    tmp = key.split(':')
    radius = float(tmp[0].split('=')[1])
    mirror_lat = float(tmp[1].split('=')[1])

    return r'$d$ = -%.1f Re, Mirroring MLAT = $%.1f^{\circ}$' % (radius, mirror_lat)

def parse_time(filename):
    s = filename.split('_')[-1][:-4]
    s = s.replace('-', ':').replace(':', '-', 2)
    return dateutil.parser.parse(s)


def prepare(filename):
    df = pd.read_csv(filename)
    df['time'] = [parse_time(f) for f in df.filename]

    colmap = {}
    for col in df.columns:
        toks = col.split('_', 1)

        if len(toks) == 1:
            continue
        tmp = toks[1].split(':')
        radius = float(tmp[0].split('=')[1])
        mirror_lat = float(tmp[1].split('=')[1])

        colmap[radius, mirror_lat] = toks[1]

    return df, colmap


def plot_K(filename):
    df, colmap = prepare(filename)

    plt.figure(figsize=(16, 6), dpi=DPI)

    items = list(colmap.items())
    items = items[::2] + items[1::2]

    for (radius, mirror_lat), key in items:
        if mirror_lat == 0:
            continue
        elif mirror_lat == 30:
            kwargs = dict(linestyle='dashed')
        else:
            kwargs = {}

        plt.plot(df.time, df['K_'+key], label=format_key(key), **kwargs)

    plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left')
    plt.ylim(0, .7)
    plt.ylabel('K')
    plt.title(format_title(filename) + f': Variation in K During {storm_name} Storm', fontweight='bold')
    plt.grid(color='#ccc', linestyle='dashed')
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b%d\n%H:%M')) 
    plt.xlim(df.time.min(), df.time.max())


def plot_LStar(filename):
    df, colmap = prepare(filename)

    plt.figure(figsize=(16, 5), dpi=DPI)

    for (radius, mirror_lat), key in colmap.items():
        if mirror_lat == 30:
            kwargs = dict(linestyle='dashed')
        else:
            kwargs = {}

        plt.plot(df.time, df['LStar_'+key], label=format_key(key), **kwargs)

    plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left')
    plt.ylabel('L* (Re)')
    plt.title(format_title(filename) + f': Variation in L* During {storm_name} Storm', fontweight='bold')
    plt.grid(color='#ccc', linestyle='dashed')
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b%d\n%H:%M'))
    plt.xlim(df.time.min(), df.time.max())
    plt.ylim(3, 16)

In [ ]:
storm_name = '2 October 2013'

In [ ]:
plot_K('LFM-20131002_RBSP_invariants.csv')
plot_K('LFMRCM-20131002_RBSP_invariants.csv')

In [ ]:
plot_LStar('LFM-20131002_RBSP_invariants.csv')
plot_LStar('LFMRCM-20131002_RBSP_invariants.csv')

In [ ]:
df1, colmap = prepare('LFM-20131002_RBSP_invariants.csv')
df2, colmap = prepare('LFMRCM-20131002_RBSP_invariants.csv')

plt.figure(figsize=(16, 5), dpi=DPI)

for (radius, mirror_lat), key in colmap.items():
    if mirror_lat == 30:
        kwargs = dict(linestyle='dashed')
    else:
        kwargs = {}

    plt.plot(df1.time, df1['LStar_'+key]  - df2['LStar_'+key], label=format_key(key), **kwargs)

plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left')
plt.ylabel('$L^*_{LFM} - L^*_{LFMRCM}$ (Re)')
plt.title(f'Variation in $\Delta$L* During {storm_name} Storm', fontweight='bold')
plt.grid(color='#ccc', linestyle='dashed')
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b%d\n%H:%M'))
plt.xlim(df1.time.min(), df1.time.max())
#plt.ylim(3, 16)

In [ ]:
cdf = cdflib.CDF('/glade/u/home/danieldas/scratch//data/LFM-20131002_GOES13/omni_hro_1min_20131001_v01.cdf')
omni_epoch = np.array(
    [dateutil.parser.parse(t) for t in cdflib.epochs.CDFepoch().encode(cdf.varget('Epoch'))]
)

fill_value = 9999.99

omni_Bx = cdf.varget('BX_GSE')
omni_Bx[omni_Bx==fill_value] = np.nan

omni_By = cdf.varget('BY_GSM')
omni_By[omni_By==fill_value] = np.nan

omni_Bz = cdf.varget('BZ_GSM')
omni_Bz[omni_Bz==fill_value] = np.nan

symh = cdf.varget('SYM_H')

fill_value = 999.99
omni_n = cdf.varget('proton_density')
omni_n[omni_n==fill_value] = np.nan

fill_value = 99999.99
omni_speed = cdf.varget('flow_speed')
omni_speed[omni_speed==fill_value] = np.nan

mask = np.isfinite(omni_Bx) & np.isfinite(omni_By) & np.isfinite(omni_Bz) & np.isfinite(omni_n) & np.isfinite(omni_speed)

omni_epoch = omni_epoch[mask]
omni_Bx = omni_Bx[mask]
omni_By = omni_By[mask]
omni_Bz = omni_Bz[mask]
omni_n = omni_n[mask]
omni_speed = omni_speed[mask]
symh = symh[mask]


In [ ]:
df, colmap = prepare('LFM-20131002_RBSP_invariants.csv')

plt.figure(figsize=(16, 5), dpi=DPI)

for (radius, mirror_lat), key in colmap.items():
    if mirror_lat == 30:
        kwargs = dict(linestyle='dashed')
    else:
        kwargs = {}

    plt.plot(df.time, df['LStar_'+key], linewidth=0, label=format_key(key), **kwargs)
leg = plt.legend(bbox_to_anchor=(1.01, 1), facecolor='white', loc='upper left', frameon=0)

for text in leg.get_texts():
    text.set_color("white")

plt.grid(color='#ccc', linestyle='dashed')
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b%d\n%H:%M'))
plt.xlim(df.time.min(), df.time.max())
plt.plot(omni_epoch, symh, color='k')
plt.ylabel('Dst (nT)')
plt.title(f'Disturbance Storm-Time Index during {storm_name} Storm', fontweight='bold')


In [ ]:
omni_epoch[np.argmin(symh)]

In [ ]:
omni_speed[np.argmin(symh)]